# Projet 4 : Analysez les ventes de votre entreprise - Nettoyage des données

**Importation des librairies**

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime

**Chargement des données**

In [2]:
clients = pd.read_csv("projet4_customers.csv")
produits = pd.read_csv("projet4_products.csv")
ventes = pd.read_csv("projet4_transactions.csv")

In [3]:
clients.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [4]:
produits.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [5]:
ventes.head()

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242


---
# <font color=blue>Mission 1 : Nettoyage des données</font>

## 1.1 Exploration du dataframe clients

In [6]:
clients.head(10)

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
5,c_415,m,1993
6,c_285,f,1967
7,c_160,f,1978
8,c_6446,m,1971
9,c_6866,m,1993


In [7]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
client_id    8623 non-null object
sex          8623 non-null object
birth        8623 non-null int64
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [8]:
clients.isna().sum()

client_id    0
sex          0
birth        0
dtype: int64

Le dataframe clients comporte 3 colonnes : client_id, sex et birth.  
Le candidat évident pour la clé primaire est client_id. Je dois vérifier qu'il n'y a pas de doublons dans cette colonne.  Toutes le valeurs semblent commencer par c_ : à vérifier car cela pourrait indiquer des lignes erronées.  
La colonne sex ne devrait prendre que deux valeurs : f ou m.  
La colonne birth indique l'année de naissance. Il faudra que je vérifie s'il n'y pas de valeur atypique sur cette colonne : personnes trop âgées ou mineures.  
Aucune des trois colonnes ne comporte de valeurs nulles. Sur ce plan, les données sont bonnes.  
Un describe devrait permettre de répondre aux interrogations ci-dessus.

In [9]:
clients.describe(include='all')

,client_id,sex,birth
count,8623,8623,8623.000000
unique,8623,2,NaN
top,c_1291,f,NaN
freq,1,4491,NaN
mean,NaN,NaN,1978.280877
std,NaN,NaN,16.919535
min,NaN,NaN,1929.000000
25%,NaN,NaN,1966.000000
50%,NaN,NaN,1979.000000
75%,NaN,NaN,1992.000000


Le dataframe clients comporte 8623 lignes, avec autant de valeurs différentes pour le client_id. Je n'ai donc pas de doublon. client_id est bien ma clé primaire.  
La colonne sex ne comporte que 2 valeurs différentes : m et f (cf head précédent). Aucun problème avec cette colonne.  
La colonne birth comporte des données comprises entre 1929 et 2004. **J'analyserai ces valeurs plus tard pour vérifier si ces valeurs extrêmes sont atypiques ou non**.

In [10]:
clients[~clients.client_id.str.startswith("c_")].describe(include='all')

,client_id,sex,birth
count,2,2,2.0
unique,2,2,NaN
top,ct_0,m,NaN
freq,1,1,NaN
mean,NaN,NaN,2001.0
std,NaN,NaN,0.0
min,NaN,NaN,2001.0
25%,NaN,NaN,2001.0
50%,NaN,NaN,2001.0
75%,NaN,NaN,2001.0


In [11]:
# seules 2 lignes ont un client_id qui ne commence pas par "c_". Je les affiche pour les vérifier.
clients[~clients.client_id.str.startswith("c_")]

,client_id,sex,birth
2735,ct_0,f,2001
8494,ct_1,m,2001


**Il se peut que ces deux valeurs soient aberrantes. A confirmer avec le dataframe ventes, et à supprimer éventuellement.**

## 1.2 Exploration du dataframe produits

In [12]:
produits.head(10)

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0
5,0_1163,9.99,0
6,1_463,36.99,1
7,0_2157,34.99,0
8,0_1915,16.99,0
9,0_389,18.99,0


In [13]:
produits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
id_prod    3287 non-null object
price      3287 non-null float64
categ      3287 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 77.1+ KB


In [14]:
produits.isna().sum()

id_prod    0
price      0
categ      0
dtype: int64

Le dataframe produits comporte également 3 colonnes.  
La colonne id_prod est une candidate naturelle à la fonction de clé primaire. Je dois vérifier si elle comporte des doublons.  
La colonne price doit être strictement positive. Il faudra également vérifier les valeurs extrêmes pour repérer d'éventuelles valeurs atypiques.  
La colonne categ semble être une colonne de catégorie. Vérifier les valeurs qu'elle peut prendre. Je remarque que la colonne id_prod semble commencer par l'identifiant de la colonne catégorie : à vérifier cette source potentielle de données erronées.  
Aucune des 3 colonnes ne comporte de valeur nulle, ce qui est une bonne choses.  
J'effectue un describe sur toutes les colonnes.

In [15]:
produits.describe(include='all')

,id_prod,price,categ
count,3287,3287.000000,3287.000000
unique,3287,NaN,NaN
top,0_2036,NaN,NaN
freq,1,NaN,NaN
mean,NaN,21.856641,0.370246
std,NaN,29.847908,0.615387
min,NaN,-1.000000,0.000000
25%,NaN,6.990000,0.000000
50%,NaN,13.060000,0.000000
75%,NaN,22.990000,1.000000


Le dataframe produits comporte 3287 lignes, avec autant de valeurs différentes pour id_prod. Il n'y a donc pas de doublon dans cette colonne, qui est bien ma clé primaire. Je vais vérifier si elle commence toujours par la valeur de categ.  
La colonne price comporte au moins une valeur aberrante avec un prix minimum négatif. Il faudra vérifier ultérieurement les valeurs atypiques, notamment vers les valeurs hautes où il y a des outliers avec un maximum égal à 300.  
La colonne categ est de type numérique et semble ne prendre que 3 valeurs : 0, 1 et 2. A vérifier avec un unique().

In [16]:
produits[produits.price <= 0].describe(include = 'all')

,id_prod,price,categ
count,1,1.0,1.0
unique,1,NaN,NaN
top,T_0,NaN,NaN
freq,1,NaN,NaN
mean,NaN,-1.0,0.0
std,NaN,NaN,NaN
min,NaN,-1.0,0.0
25%,NaN,-1.0,0.0
50%,NaN,-1.0,0.0
75%,NaN,-1.0,0.0


In [17]:
produits.categ.unique()

array([0, 1, 2], dtype=int64)

In [18]:
produits[~produits.id_prod.str.get(0).isin(produits.categ.unique().astype(str))].describe(include='all')

,id_prod,price,categ
count,1,1.0,1.0
unique,1,NaN,NaN
top,T_0,NaN,NaN
freq,1,NaN,NaN
mean,NaN,-1.0,0.0
std,NaN,NaN,NaN
min,NaN,-1.0,0.0
25%,NaN,-1.0,0.0
50%,NaN,-1.0,0.0
75%,NaN,-1.0,0.0


Je n'ai qu'une seule ligne avec un prix nul ou négatif, qui est également la seule ligne où l'id_prod (T_0) ne commence pas par la valeur de la catégorie. **Cette valeur est aberrante. Il faut la supprimer.** A priori, pas d'autres valeurs aberrantes. Il faudra vérifier les valeurs atypiques avec le dataframe ventes.  
La colonne categ ne comporte que 3 valeurs : 0, 1 et 2.

In [19]:
produits = produits[produits.price > 0]
produits.describe(include='all')

,id_prod,price,categ
count,3286,3286.000000,3286.000000
unique,3286,NaN,NaN
top,0_2036,NaN,NaN
freq,1,NaN,NaN
mean,NaN,21.863597,0.370359
std,NaN,29.849786,0.615446
min,NaN,0.620000,0.000000
25%,NaN,6.990000,0.000000
50%,NaN,13.075000,0.000000
75%,NaN,22.990000,1.000000


Il me reste à vérifier les prix atypiques. Je vérifie les valeurs qui sont considérées comme des outliers, à savoir les prix inférieurs à Q1 - 1,5*IQ et supérieurs à Q3 + 1,5*IQ.

In [20]:
prix_Q1 = produits.price.quantile(0.25)
prix_Q3 = produits.price.quantile(0.75)
prix_IQ = prix_Q3 - prix_Q1
prix_borne_inf = prix_Q1 - (1.5 * prix_IQ)
prix_borne_sup = prix_Q3 + (1.5 * prix_IQ)

In [21]:
print("Les bornes pour déterminer les outliers sont {} et {}".format(prix_borne_inf, prix_borne_sup))

Les bornes pour déterminer les outliers sont -17.009999999999998 et 46.989999999999995


Il n'y a pas d'outliers dans la partie basse car tous les prix sont positifs. Cependant Je vérifie le nombre de prix au-delà de la borne supérieure.

In [22]:
produits[produits.price > prix_borne_sup].describe(include='all')

,id_prod,price,categ
count,302,302.00000,302.000000
unique,302,NaN,NaN
top,2_18,NaN,NaN
freq,1,NaN,NaN
mean,NaN,98.48096,1.741722
std,NaN,48.05992,0.438414
min,NaN,46.99000,1.000000
25%,NaN,60.08500,1.000000
50%,NaN,84.62500,2.000000
75%,NaN,125.74000,2.000000


Il y a 302 produits avec un prix considéré comme atypique, allant de 46.99 euros à 300 euros. Rien ne me permet de conclure si ces prix sont justes ou aberrants.  
  
Je pousse plus loin l'analyse en regardant la fonction de répartition des prix en deux temps car les prix sont très variés entre le Q3 et le max. Je divise les données des 3 premiers quartiles (de min à Q3) en 20 segments de taille identique. Et la répartion des données du dernier quartile est analysée à part avec 20 nouveaux segments identiques.

In [23]:
effectifs = produits[produits.price <= 22.99].price.value_counts(bins=20)
temp = pd.DataFrame(effectifs.index, columns=["price"])
temp["n"] = effectifs.values
temp["f"] = effectifs.values / len(produits[produits.price <= 22.99])
temp = temp.sort_values("price")
temp["F"] = temp["f"].cumsum()
temp

,price,n,f,F
19,"(0.597, 1.738]",60,0.024106,0.024106
11,"(1.738, 2.857]",105,0.042186,0.066292
3,"(2.857, 3.975]",153,0.061470,0.127762
0,"(3.975, 5.094]",262,0.105263,0.233025
5,"(5.094, 6.212]",145,0.058256,0.291282
2,"(6.212, 7.331]",161,0.064685,0.355966
6,"(7.331, 8.449]",144,0.057855,0.413821
7,"(8.449, 9.568]",141,0.056649,0.470470
8,"(9.568, 10.686]",136,0.054640,0.525110
4,"(10.686, 11.805]",151,0.060667,0.585777


Les prix sont assez uniformément répartis entre les 20 segments dans les trois premiers quartiles. Cela semble cohérent.

In [24]:
effectifs = produits[produits.price > 22.99].price.value_counts(bins=20)
temp = pd.DataFrame(effectifs.index, columns=["price"])
temp["n"] = effectifs.values
temp["f"] = effectifs.values / len(produits[produits.price > 22.99])
temp = temp.sort_values("price")
temp["F"] = temp["f"].cumsum()
temp

,price,n,f,F
0,"(22.782, 36.907]",395,0.495609,0.495609
1,"(36.907, 50.754]",129,0.161857,0.657465
2,"(50.754, 64.601]",68,0.085320,0.742785
3,"(64.601, 78.448]",37,0.046424,0.789210
4,"(78.448, 92.295]",34,0.042660,0.831870
5,"(92.295, 106.142]",30,0.037641,0.869511
6,"(106.142, 119.989]",23,0.028858,0.898369
7,"(119.989, 133.836]",19,0.023839,0.922208
8,"(133.836, 147.683]",16,0.020075,0.942284
9,"(147.683, 161.53]",14,0.017566,0.959849


Dans la partie haute des prix (supérieure à Q3), les deux tiers des produits sont compris dans les deux premiers segments et ont un prix inférieur à 50 euros. Les produits dont le prix est supérieur à 150 euros ne représentent que 5% des produits dans la partie haute des prix.

Rien ne permet d'affirmer avec certitude que certains prix sont aberrants ou erronés.  
De façon générale, on peut supposer que la catégorie 0 correspond aux livres de poche. Les prix les plus bas peuvent correspondre à des accessoires comme des marque-pages, à des promotions ou à des livres audio si le site s'est lancé sur ce segment.  
La catégorie 1 peut correspondre aux livres brochés.  
Enfin la catégorie 2 peut correspondre aux livres professionnels, techniques, spécialisés. Voire aussi aux livres d'art. Ces catégories de livres coûtent en général assez cher.  
Pour les prix les plus bas de la catégorie 0 et les plus élevés de la catégorie 2, il convient de vérifier leur exactitude en se renseignant au sein de l'entreprise.

## 1.3 Exploration du dataframe ventes

In [25]:
ventes.head(10)

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242
5,0_1085,2021-09-15 05:47:48.215162,s_90139,c_2526
6,0_1508,2021-08-29 05:39:01.055455,s_82100,c_5799
7,0_1627,2021-09-19 14:54:52.410362,s_92294,c_1422
8,0_1469,2022-02-03 07:13:22.559995,s_159252,c_2207
9,0_1453,2022-02-26 09:03:10.571665,s_171098,c_5433


In [26]:
ventes.isna().sum()

id_prod       0
date          0
session_id    0
client_id     0
dtype: int64

In [27]:
ventes.describe(include='all')

,id_prod,date,session_id,client_id
count,337016,337016,337016,337016
unique,3266,336855,169195,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,1081,13,200,12855


In [28]:
ventes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
id_prod       337016 non-null object
date          337016 non-null object
session_id    337016 non-null object
client_id     337016 non-null object
dtypes: object(4)
memory usage: 10.3+ MB


Il y a 4 colonnes, toutes de type string, et 337016 lignes dans le dataframe ventes. Le dataframe ne comporte aucune valeur nulle, ce qui est bon signe quant à sa qualité.  
Aucune colonne ne comporte autant de valeurs uniques qu'il y a de lignes. La clé primaire est donc composée de plusieurs champs. Une session correspond à une commande. Sur une même session, un même client peut commander plusieurs livres. La combinaison logique pour la clé primaire me semble session_id et id_prod.  
Le champ date sera à retraiter pour le passer en numérique de type date. **Je remarque que la valeur top de date commence par 'test'. Il faut vérifier la présence de lignes de test sur chacune des colonnes.** De même la session_id a une valeur top s_0 qui pourrait correspondre à une ligne de test.  
**Je remarque la présence de id_prod et de client_id qui permettent de faire des jointures pour avoir toutes les informations sur les ventes, les produits et les clients dans un même dataframe.** La jointure externe à gauche, avec le dataframe ventes en référence (donc à gauche), garantit de garder les informations pour toutes les ventes. Il faudra vérifier si l'on dispose d'informations pour tous les produits et tous les clients, ou si les jointures introduisent des valeurs manquantes.  
  
Pour commencer, je vérifie les lignes de la colonne date qui comprennent des valeurs alphabétiques. Normalement, elles ne devraient contenir que des valeurs numériques, et des caractères comme des tirets, des points, des doubles points ou des espaces.

In [29]:
ventes[ventes.date.str.contains('[a-zA-Z]')].describe(include='all')

,id_prod,date,session_id,client_id
count,200,200,200,200
unique,1,39,1,2
top,T_0,test_2021-03-01 02:30:02.237413,s_0,ct_0
freq,200,13,200,106


Il y a 200 lignes qui comprennent des caractères alphabétiques dans la colonne date.  
Elles portent toutes sur le produit d'id_prod T_0, que nous avons déjà identifié comme ayant un prix négatif et que nous avons effacé du dataframe produits.  
Elles ont toutes la même session_id s_0.  
Elles ont 39 dates différentes et 2 client_id différents. Vérifions ces clients id et le nombre de fois où ils sont présents. Je vérifie aussi le nombre de fois où l'id_prod T_0 est présent, pour m'assurer qu'il n'est présent que dans les sessions de test. Idem pour s_0.

In [30]:
ventes[ventes.date.str.contains('[a-zA-Z]')].client_id.unique()

array(['ct_1', 'ct_0'], dtype=object)

In [31]:
ventes[(ventes.client_id == 'ct_0') | (ventes.client_id == 'ct_1')].count()

id_prod       200
date          200
session_id    200
client_id     200
dtype: int64

In [32]:
ventes[ventes.id_prod == 'T_0'].count()

id_prod       200
date          200
session_id    200
client_id     200
dtype: int64

In [33]:
ventes[ventes.session_id == 's_0'].count()

id_prod       200
date          200
session_id    200
client_id     200
dtype: int64

Nous remarquons que :  
- les sessions de test prennent les deux valeurs atypiques décelées dans le dataframe clients 'ct_0' et 'ct_1' ;
- ces deux valeurs atypiques concernent uniquement les sessions de test (nous ne les retrouvons que sur 200 lignes). **Je peux donc supprimer les clients dont l'id est 'ct_0' et 'ct_1'** ;
- de même, nous ne retrouvons les id_prod 'T_0' et les session_id 's_0' que sur 200 lignes. Ces 2 valeurs ne concernent que les sessions de test. Cela confirme le bien fondé de la suppression du produit avec un prix négatif dans le dataframe produits. **Je peux supprimer les ventes qui appartiennent à la session 's_0'**.

In [34]:
clients = clients[(clients.client_id != 'ct_0') & (clients.client_id != 'ct_1')]
ventes = ventes[ventes.session_id != 's_0']

In [35]:
ventes.describe(include='all')

,id_prod,date,session_id,client_id
count,336816,336816,336816,336816
unique,3265,336816,169194,8600
top,1_369,2022-01-21 12:49:30.029159,s_118668,c_1609
freq,1081,1,14,12855


Je remarque que j'ai à peu près deux fois plus de valeurs distinctes pour le champ date que pour le champ session_id :  
- soit le champ date enregistre le moment où un produit est ajouté au panier, ce qui signifierait qu'une session aboutissant à une vente comprend en moyenne à peu près deux produits dans le panier ;  
- soit la quasi-totalité des lignes sont en doublon avec deux dates différentes pour une même session.  
  
Dans l'éventualité du premier cas, le champ date constituerait une autre clé primaire candidate car nous avons autant de dates que de lignes.  
  
Pour commencer, j'analyse le champ date pour les dix premières sessions.

In [36]:
ventes.sort_values(by="session_id").iloc[:10]

,id_prod,date,session_id,client_id
176588,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
309544,1_635,2021-03-01 00:10:33.163037,s_10,c_2218
42033,0_1451,2021-03-01 04:43:58.025677,s_100,c_3854
148924,0_1030,2021-03-01 04:12:43.572994,s_100,c_3854
328736,0_1590,2021-03-03 02:49:03.169115,s_1000,c_1014
250061,0_1625,2021-03-03 02:38:09.568389,s_1000,c_1014
214607,0_1438,2021-03-03 03:25:12.320225,s_1000,c_1014
202629,0_1449,2021-03-03 03:18:58.492283,s_1000,c_1014
75293,1_395,2021-03-22 17:46:05.819130,s_10000,c_476
6546,0_1418,2021-03-22 18:15:03.831240,s_10000,c_476


Les valeurs de session_id font penser à la présence d'autres lignes de test. J'élargis l'observation aux 25 premières lignes en fonction de session_id.

In [37]:
ventes.sort_values(by="session_id").iloc[:25]

,id_prod,date,session_id,client_id
176588,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
309544,1_635,2021-03-01 00:10:33.163037,s_10,c_2218
42033,0_1451,2021-03-01 04:43:58.025677,s_100,c_3854
148924,0_1030,2021-03-01 04:12:43.572994,s_100,c_3854
328736,0_1590,2021-03-03 02:49:03.169115,s_1000,c_1014
250061,0_1625,2021-03-03 02:38:09.568389,s_1000,c_1014
214607,0_1438,2021-03-03 03:25:12.320225,s_1000,c_1014
202629,0_1449,2021-03-03 03:18:58.492283,s_1000,c_1014
75293,1_395,2021-03-22 17:46:05.819130,s_10000,c_476
6546,0_1418,2021-03-22 18:15:03.831240,s_10000,c_476


Le format de la colonne session_id ne permet de mener une analyse facilement, car le tri selon ce critère ne s'effectue pas correctement.  
Je vais transformer ma colonne session_id en rajoutant des 0 après le tiret et avant le nombre si nécessaire pour toujours avoir le même nombre de chiffres dans une session id. Cela permettra de trier efficacement les sessions selon leur id.  
je vais également convertir ma colonne date en type date.

In [38]:
# je vérifie les valeurs maximum pour savoir combien de chiffres je dois avoir dans session-id
ventes.sort_values(by="session_id", ascending=False).iloc[:10]

,id_prod,date,session_id,client_id
143919,0_1411,2021-10-04 18:50:59.099942,s_99998,c_2795
251881,0_1572,2021-10-04 18:45:54.374885,s_99998,c_2795
8129,0_1604,2021-10-04 19:00:20.030388,s_99998,c_2795
91103,0_1197,2021-10-04 18:45:38.003516,s_99997,c_3521
195828,0_1475,2021-10-04 18:45:05.640783,s_99996,c_4900
38496,0_2231,2021-10-04 18:38:36.333661,s_99996,c_4900
77106,0_1609,2021-10-04 18:39:10.485474,s_99996,c_4900
32376,0_1362,2021-10-04 18:58:49.228317,s_99996,c_4900
104067,0_1784,2021-10-04 18:35:32.201073,s_99995,c_4170
319185,0_1636,2021-10-04 18:56:23.112236,s_99994,c_7685


In [39]:
# j'ai une session_id max supérieure à s_100000 mais inférieure à s_999999. 6 chiffres sont donc suffisants.
ventes.session_id = ventes.session_id.str[:2] + ventes.session_id.str[2:].str.zfill(6)
ventes.date = pd.to_datetime(ventes.date)

In [40]:
ventes.head()

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_018746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_094290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_063642,c_1242


In [41]:
ventes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336816 entries, 0 to 337015
Data columns (total 4 columns):
id_prod       336816 non-null object
date          336816 non-null datetime64[ns]
session_id    336816 non-null object
client_id     336816 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 12.8+ MB


In [42]:
ventes.sort_values(by="session_id").iloc[:25]

,id_prod,date,session_id,client_id
176588,0_1259,2021-03-01 00:01:07.843138,s_000001,c_329
95285,0_1390,2021-03-01 00:02:26.047414,s_000002,c_664
21078,0_2245,2021-03-01 00:09:29.301897,s_000003,c_580
135986,0_1110,2021-03-01 00:38:57.630675,s_000003,c_580
157168,0_1638,2021-03-01 00:10:37.223732,s_000003,c_580
216737,0_1352,2021-03-01 00:02:38.311413,s_000003,c_580
63764,1_310,2021-03-01 00:17:11.089942,s_000004,c_7912
180767,0_1458,2021-03-01 00:04:54.559692,s_000004,c_7912
293481,0_1358,2021-03-01 00:05:18.801198,s_000005,c_2033
19554,0_1475,2021-03-01 00:16:16.649539,s_000006,c_4908


Il semble que nous n'avons plus de lignes de test et que la date indique le moment où le produit est ajouté au panier. De ce fait, la colonne date est la clé primaire.  
La combinaison session_id et id_prod est une autre clé possible, mais elle n'est pas primaire car elle comporte plus d'attributs. Et elle comporte un risque : que le même produit soit ajouté plusieurs fois au panier lors d'une même session.  
J'effectue une dernière vérification pour valider définitivement ce choix : le nombre moyen de produits acheté lors d'une session doit être légèrement inférieur à 2.

In [43]:
temp = ventes.groupby("session_id").count()

In [44]:
temp.mean()

id_prod      1.990709
date         1.990709
client_id    1.990709
dtype: float64

L'hypothèse sur le champ date est bien validée.

## 1.4 Création et exploration du dataframe data par jointures

In [45]:
data = ventes.merge(produits, how="left", on="id_prod")
data = data.merge(clients, how="left", on="client_id")
data.head()

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1483,2021-04-10 18:37:28.723910,s_018746,c_4450,4.99,0.0,f,1977
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,65.75,2.0,f,2000
2,1_374,2021-09-23 15:13:46.938559,s_094290,c_4270,10.71,1.0,f,1979
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597,4.20,0.0,m,1963
4,0_1351,2021-07-17 20:34:25.800563,s_063642,c_1242,8.99,0.0,f,1980


Je change l'ordre des colonnes du dataframe data pour plus de lisibilité.
Je vérifie que les données sont disponibles pour tous les clients et tous les produits, en regardant si des valeurs nulles sont apparues.

In [46]:
data = data[["session_id","date","id_prod","price","categ","client_id","sex","birth"]]
data.isna().sum()

session_id      0
date            0
id_prod         0
price         103
categ         103
client_id       0
sex             0
birth           0
dtype: int64

Je constate que 103 lignes n'ont pas d'information de prix et de catégorie. Je regarde si beaucoup de produits différents sont concernés par ce manque d'information.

In [47]:
data[data.price.isna()].id_prod.unique()

array(['0_2245'], dtype=object)

Un seul produit n'est pas renseigné, soit 103 lignes sur un total de 336 816 lignes. Ce produit représente une partie infime des ventes : moins de 0,05% des données. Je pourrais donc décider de supprimer ces lignes.  
Cependant, j'ai déjà deux informations : l'id_prod 0_2245 et par conséquent la catégorie qui est 0.  
Il reste à choisir le prix que nous allons appliquer à ce produit. Je limite l'analyse aux prix des produits de la catégorie 0 pour me baser sur les produits les plus comparables au produit manquant.

In [48]:
produits[produits.categ == 0].describe(include='all')

,id_prod,price,categ
count,2308,2308.000000,2308.0
unique,2308,NaN,NaN
top,0_367,NaN,NaN
freq,1,NaN,NaN
mean,NaN,11.732795,0.0
std,NaN,7.565755,0.0
min,NaN,0.620000,0.0
25%,NaN,5.587500,0.0
50%,NaN,10.320000,0.0
75%,NaN,16.655000,0.0


La médiane des prix de la catégorie 0 est 10,32 euros alors que la moyenne est 11,73 euros. Je remarque que la valeur max des prix (40,99 euros) est 'anormalement' supérieure au troisième quartile. Alors que 5 à 6 euros séparent la valeur min du premier quartile, ainsi que le premier quartile de la médiane, et la médiane du troisième quartile, plus de 20 euros séparent le troisème quartile de la valeur max. Ces valeurs maximales très élevées, que l'on peut considérer comme atypique, tirent la moyenne vers le haut. Par conséquent, la valeur médiane est plus cohérente comme prix de substitution.  
**J'ajoute ce produit au dataframe produits. Je reconstruis le dataframe data pour inclure ces nouvelles données.**

In [49]:
temp = pd.DataFrame([['0_2245', produits[produits.categ==0].price.median(), 0]])
temp.columns = ["id_prod", "price", "categ"]
temp

,id_prod,price,categ
0,0_2245,10.32,0


In [50]:
produits = pd.concat([produits, temp], ignore_index=True)
produits.tail()

,id_prod,price,categ
3282,0_146,17.14,0
3283,0_802,11.22,0
3284,1_140,38.56,1
3285,0_1920,25.16,0
3286,0_2245,10.32,0


In [51]:
data = ventes.merge(produits, how="left", on="id_prod")
data = data.merge(clients, how="left", on="client_id")
#je ré-organise les colonnes
data = data[["session_id","date","id_prod","price","categ","client_id","sex","birth"]]
data.head()

,session_id,date,id_prod,price,categ,client_id,sex,birth
0,s_018746,2021-04-10 18:37:28.723910,0_1483,4.99,0,c_4450,f,1977
1,s_159142,2022-02-03 01:55:53.276402,2_226,65.75,2,c_277,f,2000
2,s_094290,2021-09-23 15:13:46.938559,1_374,10.71,1,c_4270,f,1979
3,s_105936,2021-10-17 03:27:18.783634,0_2186,4.20,0,c_4597,m,1963
4,s_063642,2021-07-17 20:34:25.800563,0_1351,8.99,0,c_1242,f,1980


In [52]:
data.isna().sum()

session_id    0
date          0
id_prod       0
price         0
categ         0
client_id     0
sex           0
birth         0
dtype: int64

## 1.5 Création et exploration du dataframe panier

Les dataframes ventes et, par extension, data comprennent une ligne par produit ajouté dans le panier. Des informations regroupées par panier seront plus pertinentes pour une analyse des ventes. Je décide donc de créer un dataframe panier, basé sur data, qui comprend les colonnes suivantes :
- session id : c'est mon attribut de partitionnement ;
- date d'achat : je considère que l'achat est réalisé lors de l'ajout du dernier produit dans le panier - n'est significatif que si le nombre de produits est > 1 ;
- durée de session : c'est la différence de temps entre le moment où les moments d'ajout au panier du premier et du dernier produit ;
- nombre de produits dans le panier : nombre de produits achetés lors de cette session - calculé grâce au nombre de dates différentes au cas où le même id_prod soit ajouté plusieurs fois au même panier ;
- nombre de produits différents : permet de savoir si un panier contient plusieurs fois un même produit (si le nombre de produits différents n'est pas égal au nombre de produits dans le panier) ;
- montant du panier = somme des prix des produits dans le panier ;
- client id ;
- sex ;
- birth ;
- client age : je pourrais calculer cet page sur la base de la date d'achat et de l'année de naissance, malgré l'approximation liée au fait que nous n'avons que l'année de naissance. Cependant, pour qu'un même client soit identifié avec le même âge dans l'analyse, nous allons calculer l'âge des clients en faisant la différence entre l'année 2022 (année où l'étude est menée) et l'année de naissance ;
- year month : colonne qui indique le mois et l'année de l'achat, pour des regroupements et analyses ultérieures ;
- day of week : colonne qui indique le jour de l'achat, pour des regroupement et analyses ultérieures.  
  
Les colonnes client_age, year_month et day_of_week sont ensuite ajoutées par jointure au dataframe data, sur la base de la colonne session_id. J'ajoute également client_age au dataframe clients.

In [53]:
# je vérifie qu'une session ne corresponde qu'à un seul client
temp = data.groupby("session_id")["client_id"].nunique()
temp.max()

1

In [54]:
# remarque : je crée un dataframe auquel j'ajoute des colonnes grâce à un dataframe temporaire pour chaque opération
# cela augmente considérablement le temps de calcul par rapport à l'utilisation de apply
panier = data.groupby(["session_id","client_id","sex","birth"])[["date"]].max()
panier.columns = ["date_achat"]
temp = data.groupby(["session_id","client_id","sex","birth"])["date"].min()
panier["duree_session"] = panier.date_achat - temp
panier["nombre_produits_panier"] = data.groupby(["session_id","client_id","sex","birth"])["date"].count()
panier["nombre_produits_differents"] = data.groupby(["session_id","client_id","sex","birth"])["id_prod"].nunique()
panier["montant_panier"] = data.groupby(["session_id","client_id","sex","birth"])["price"].sum()

In [55]:
# je vérifie si la colonne nombre_produits_differents est utile
panier[panier.nombre_produits_panier != panier.nombre_produits_differents].count()

date_achat                    408
duree_session                 408
nombre_produits_panier        408
nombre_produits_differents    408
montant_panier                408
dtype: int64

In [56]:
panier.reset_index(inplace=True)
panier.head()

,session_id,client_id,sex,birth,date_achat,duree_session,nombre_produits_panier,nombre_produits_differents,montant_panier
0,s_000001,c_329,f,1967,2021-03-01 00:01:07.843138,00:00:00,1,1,11.99
1,s_000002,c_664,m,1960,2021-03-01 00:02:26.047414,00:00:00,1,1,19.37
2,s_000003,c_580,m,1988,2021-03-01 00:38:57.630675,00:36:19.319262,4,4,24.99
3,s_000004,c_7912,f,1989,2021-03-01 00:17:11.089942,00:12:16.530250,2,2,20.75
4,s_000005,c_2033,f,1956,2021-03-01 00:05:18.801198,00:00:00,1,1,16.49


In [57]:
panier.isna().sum()

session_id                    0
client_id                     0
sex                           0
birth                         0
date_achat                    0
duree_session                 0
nombre_produits_panier        0
nombre_produits_differents    0
montant_panier                0
dtype: int64

In [58]:
panier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169194 entries, 0 to 169193
Data columns (total 9 columns):
session_id                    169194 non-null object
client_id                     169194 non-null object
sex                           169194 non-null object
birth                         169194 non-null int64
date_achat                    169194 non-null datetime64[ns]
duree_session                 169194 non-null timedelta64[ns]
nombre_produits_panier        169194 non-null int64
nombre_produits_differents    169194 non-null int64
montant_panier                169194 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(3), object(3), timedelta64[ns](1)
memory usage: 11.6+ MB


Le dataframe panier semble avoir des données correctes, ce qui est logique étant donné qu'il est basé sur le dataframe data qui a déjà été nettoyé. Je peux ajouter les nouvelles colonnes.

In [59]:
panier["client_age"] = 2022 - panier.birth
panier["year_month"] = (panier.date_achat.dt.year * 100 + panier.date_achat.dt.month).astype(str)
panier["day_of_week"] = panier.date_achat.dt.day_name()

In [60]:
data = data.merge(panier[["session_id","client_age","year_month","day_of_week"]], how="left", on="session_id")
clients["age"] = 2022 - clients.birth

In [61]:
panier.describe(include='all')

,session_id,client_id,sex,birth,date_achat,duree_session,nombre_produits_panier,nombre_produits_differents,montant_panier,client_age,year_month,day_of_week
count,169194,169194,169194,169194.000000,169194,169194,169194.000000,169194.000000,169194.000000,169194.000000,169194,169194
unique,169194,8600,2,NaN,169194,NaN,NaN,NaN,NaN,NaN,12,7
top,s_037425,c_1609,m,NaN,2021-09-29 00:41:06.303301,NaN,NaN,NaN,NaN,NaN,202112,Monday
freq,1,5501,85556,NaN,1,NaN,NaN,NaN,NaN,NaN,15564,24519
first,NaN,NaN,NaN,NaN,2021-03-01 00:01:07.843138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2022-02-28 23:59:58.040472,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,1976.131512,NaN,0 days 00:13:46.953406,1.990709,1.988292,34.266231,45.868488,NaN,NaN
std,NaN,NaN,NaN,14.876864,NaN,0 days 00:18:20.758984,1.274256,1.271262,31.872258,14.876864,NaN,NaN
min,NaN,NaN,NaN,1929.000000,NaN,0 days 00:00:00,1.000000,1.000000,0.620000,18.000000,NaN,NaN
25%,NaN,NaN,NaN,1967.000000,NaN,0 days 00:00:00,1.000000,1.000000,15.200000,36.000000,NaN,NaN


Je vérifie la pertinence des valeurs contenues dans le dataframe panier avec celles de du dataframe data en calculant le CA total obtenu dans les 2 dataframes, ainsi que le nombre totale de produits vendus.

In [62]:
if (data.price.sum() == panier.montant_panier.sum()):
    print("CA total OK")
else:
    print("Erreur dans le CA total")

if (len(data) == panier.nombre_produits_panier.sum()):
    print("Nombre total de produits vendus OK")
else:
    print("Erreur dans le nombre total de produits vendus")    

CA total OK
Nombre total de produits vendus OK


## 1.6 Sauvegarde des dataframes pour l'analyse

In [63]:
clients.to_csv("projet04_clients_clean.csv", index=False)
produits.to_csv("projet04_produits_clean.csv", index=False)
ventes.to_csv("projet04_ventes_clean.csv", index=False)
data.to_csv("projet04_data_clean.csv", index=False)
panier.to_csv("projet04_panier_clean.csv", index=False)